In [1]:
''' Day 5_1 '''

' Day 5_1 '

In [167]:
import docx
import pandas as pd

In [168]:
def read_docx(filename):
    document=docx.Document(filename)
    #解析文本
    paragraphs = []
    for p in document.paragraphs:
        paragraphs.append(p.text)
    #解析表格
    tables = []
    for table in document.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
    # 首行可以设置成列名                
    # tables.append(pd.DataFrame(df).rename(columns=tables[0].loc[0]).drop(0))
    tables.append(pd.DataFrame(df))
    return paragraphs, tables

In [169]:
paragraphs, tables = read_docx('demo.docx')

In [170]:
paragraphs

['Document Title',
 'A plain paragraph having some bold and some italic.',
 'Heading, level 1',
 'first item in unordered list',
 'first item in ordered list',
 '',
 '\n']

In [171]:
tables[0]

,0,1,2
0,Qty,Id,Desc
1,3,101,Spam
2,7,422,Eggs
3,4,631,"Spam, spam, eggs, and spam"


In [175]:
df=tables[0].rename(columns=tables[0].loc[0]).drop(0)

In [176]:
df

,Qty,Id,Desc
1,3,101,Spam
2,7,422,Eggs
3,4,631,"Spam, spam, eggs, and spam"


In [165]:
def read_imgs(filename, save_path):
    """get all of the images in a paragraph 
    :param par: a paragraph object from docx
    :return: a list of r:embed 
    """
    import xml.etree.cElementTree as ET
    import re
    
    document = docx.Document(filename)
    proxy=[]
    for p in document.paragraphs:
        proxy.append(p._element.xml)
    
    rIds = []
    namespace = {
             'a':"http://schemas.openxmlformats.org/drawingml/2006/main", \
             'r':"http://schemas.openxmlformats.org/officeDocument/2006/relationships", \
             'wp':"http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing"}
    
    for p in proxy:
        root = ET.fromstring(p)
        inlines = root.findall('.//wp:inline',namespace)
        if inlines is not None:
            for inline in inlines:
                imgs = inline.findall('.//a:blip', namespace)
                for img in imgs: 
                    id = img.attrib['{{{0}}}embed'.format(namespace['r'])]
                    rIds.append(id)
    
    # 保存图片
    save_img_i = 1
    for rid in rIds:
        f=open(save_path+"img%d.jpg" % save_img_i,'wb')
        f.write(document.part.related_parts[rid].blob)
        f.close()
        save_img_i+=1
    
    return rIds

In [166]:
rIds = read_imgs(filename = 'demo(复件).docx', save_path = 'img/')